<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Разбивка-на-выборки:-обучающую-и-валидационную" data-toc-modified-id="Разбивка-на-выборки:-обучающую-и-валидационную-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разбивка на выборки: обучающую и валидационную</a></span></li><li><span><a href="#Обучение-модели,-получение-предсказаний-на-валидационной-выборке" data-toc-modified-id="Обучение-модели,-получение-предсказаний-на-валидационной-выборке-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение модели, получение предсказаний на валидационной выборке</a></span></li><li><span><a href="#Расчет-среднего-запаса-предсказанного-сырья-и-RMSE-модели" data-toc-modified-id="Расчет-среднего-запаса-предсказанного-сырья-и-RMSE-модели-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Расчет среднего запаса предсказанного сырья и RMSE модели</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li></ul></div>

# Проект: "Выбор локации для скважины"

**Задача проекта**: 

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Условия задачи:

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

В рамках проекта мы проведем следующие **этапы исследования**: 

1) Ознакомимся с данными и подготовим их для дальнейшего исследования;

2) Обучим модель;

3) Произведем расчеты прибыли и рисков;

4) Выберем лучший регион для бурения новой скважины.

Приступим к исследованию! 

## Загрузка и подготовка данных

In [13]:
#импортируем необходимые для проекта библиотеки и алгоритмы:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [14]:
#прочтем файлы с данными геологоразведки трёх регионов, выведем на экран первые 5 строк каждой таблицы для ознакомления: 
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')
display('data_1:', data_1.head())
display('data_2:', data_2.head())
display('data_3:', data_3.head())

'data_1:'

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


'data_2:'

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


'data_3:'

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Поясним, какие данные содержатся в таблице.

*Признаки*:

f0, f1, f2 — три признака точек (значимые признаки);
product — объём запасов в скважине (тыс. баррелей).

*Целевой признак*: 

product — объём запасов в скважине (тыс. баррелей).

In [15]:
#выведем на экран общую информацию о данных таблиц: 
display('data_1', data_1.info())
display('data_2', data_2.info())
display('data_3', data_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


'data_1'

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


'data_2'

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


'data_3'

None

Исходя из общей информации мы видим, что пропусков в данных нет, типы данных указаны верно. 

Таким образом, данные готовы к исследованию. 

In [16]:
!pip install pandas_profiling==1.4.1 -q
!pip install -U scikit-learn -q

In [17]:
import  pandas_profiling
pandas_profiling.ProfileReport(data_1)

/home/jovyan/.local/lib/python3.7/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,5
Number of observations,100000
Total Missing (%),0.0%
Total size in memory,3.8 MiB
Average record size in memory,40.0 B
Numeric,4
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


## Обучение и проверка модели

### Разбивка на выборки: обучающую и валидационную

Разобьем данные на обучающую и валидационную выборки в соотношении 75:25. 

Чтобы не повторять однотипые действия для каждой таблицы, запишем функцию для получения выборок. 

In [18]:
#запишем функцию для получения выборок, которая на вход принимает датасет, а на выходе дает обуч. и валид. выборки: 
def split(data):
    features = data[['f0','f1','f2']]
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=1000000, shuffle=True)
    return features_train, features_valid, target_train, target_valid

In [19]:
#с помощью функции получим обучающие и валидационные выборки для каждой таблицы: 
features_train_1, features_valid_1, target_train_1, target_valid_1 = split(data_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = split(data_2)
features_train_3, features_valid_3, target_train_3, target_valid_3 = split(data_3)
#для проверки выведем на экран размеры получившихся выборок из 1 таблицы методом shape: 
display('Размер features и target обучающей выборки:', features_train_1.shape, target_train_1.shape)
display('Размер features и target валидационной выборки:', features_valid_1.shape, target_valid_1.shape)

'Размер features и target обучающей выборки:'

(75000, 3)

(75000,)

'Размер features и target валидационной выборки:'

(25000, 3)

(25000,)

Размеры получившихся выборок говорят о том, что разбивка данных проведена успешно. 

### Обучение модели, получение предсказаний на валидационной выборке

Для дальнейших расчетов нам необходимо обучить модель и получить предсказания на валидационной выборке. 

Запишем функцию для исключения однотипных действий. 

In [20]:
#запишем функцию, с помощью которой обучим модель линейной регрессии и получим предсказания модели: 
def predict_linear_regression(features_train, target_train, features_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    return predict

In [21]:
#с помощью нашей функции получим предсказания для каждого набора данных: 
predict_valid_1 = predict_linear_regression(features_train_1, target_train_1, features_valid_1)
predict_valid_2 = predict_linear_regression(features_train_2, target_train_2, features_valid_2)
predict_valid_3 = predict_linear_regression(features_train_3, target_train_3, features_valid_3)

### Расчет среднего запаса предсказанного сырья и RMSE модели

Теперь для каждого из 3 наборов данных рассчитаем средний запас предсказанного сырья, а также корень из средней квадратичной ошибки. 
Как и ранее, используем функцию: 

In [22]:
#запишем функцию,которая на вход принимает предсказания и значения целевого признака, на выходе- средний запас предсказанного сырья и RMSE:
def mean_and_rmse(target, predict): 
    mean = predict.mean()
    rmse = mean_squared_error(target, predict) ** 0.5
    return display (f'Cредний запас предсказанного сырья - {mean}, RMSE модели - {rmse}')

In [23]:
#выведем на экране средний запас предсказанного сырья и RMSE модели для каждого региона:
display('Регион 1:')
mean_and_rmse(target_valid_1, predict_valid_1)
display('Регион 2:')
mean_and_rmse(target_valid_2, predict_valid_2)
display('Регион 3:')
mean_and_rmse(target_valid_3, predict_valid_3)

'Регион 1:'

'Cредний запас предсказанного сырья - 92.63659883743223, RMSE модели - 37.71640470180887'

'Регион 2:'

'Cредний запас предсказанного сырья - 68.76434676065062, RMSE модели - 0.8858458984188471'

'Регион 3:'

'Cредний запас предсказанного сырья - 95.03211087480763, RMSE модели - 40.02824634351358'

**Вывод**

После обучения моделей и получения предсказаний объема сырья для каждого региона, мы получили следующие результаты: 

- для первого региона средний запас предсказанного сырья- 92.6 тыс. баррелей, при этом возможны отклонения от среднего на 37.7 тыс. баррелей (rmse); 
- для второго региона 68.7 и 0.88 тыс. баррелей соответственно; 
- для третьего- 95 и 40. 

По этми данным нельзя с уверенностью сказать, какой из регионов следует выбрать с точки зрения прибыльности, так как в первом и третьем случае мы наблюдаем довольно большое значение метрики rmse, во втором же средние запасы предсказанного сырья гораздо ниже, чем в первом и третьем регионе. 

## Подготовка к расчёту прибыли

In [24]:
#сохраним все ключевые значения для расчетов в отдельных переменных: 
EXPENSES = 10**7 #бюджет на разработку скважин в регионе (тыс. руб.)
INCOME_UNIT = 450 #доход с каждой единицы продукта(тыс.руб.)
CRITICAL_LOSS_PROBABILITY = 0.025 #критичная вероятность убытков -  2,5 % (в долях)
NUMBER_EXPLORATION = 500 #количество скважин для исследования
NUMBER_DEVELOPED = 200 #количество скважин для разработки

In [25]:
#рассчитаем достаточный объём сырья для безубыточной разработки новой скважины:
break_even = EXPENSES / (INCOME_UNIT * NUMBER_DEVELOPED)
display("Достаточный объём сырья для безубыточной разработки новой скважины (точка безубыточности)=", break_even)

'Достаточный объём сырья для безубыточной разработки новой скважины (точка безубыточности)='

111.11111111111111

**Вывод**

Мы подготовили данные для расчета прибыли и рассчитали точку безубыточности для разработки новой скважины. 
Чтобы покрыть затраты на разработку одной скважины, необходимо, чтобы объем запасов в скважине был не менее = 111.11 тыс. баррелей нефти.  Этот показатель выше средних значений предсказанного сырья по всем 3 регионам. 

Уже на этом этапе мы можем предположить, что регион №2 не выйдет в лидеры, посколько среднее значение гораздо ниже 111 (почти в 2 раза). Для того чтобы сделать обоснованные выводы, произведем расчет прибыли и рисков с использованием техники bootstrap.

## Расчёт прибыли и рисков 

Посчитаем прибыль и риски для каждого региона с использованием техники bootstrap.

Чтобы не повторять код многократно, запишем функцию, которая на вход принимает предсказания модели, значения целевого признака, а на выходе даст значения средней прибыли, 95% доверительного интервала и риска убытков. 

In [39]:
def function (target, predict):
    state = np.random.RandomState(1000000)
    revenue = []
    for i in range(1000):
        target_subsample = pd.Series(target).reset_index(drop=True).sample(n=NUMBER_EXPLORATION, replace=True, random_state=state)
        predict_subsample = pd.Series(predict)[target_subsample.index]
        predict_sorted = predict_subsample.sort_values(ascending=False)
        selected_target = target_subsample[predict_sorted.index][:NUMBER_DEVELOPED]
        profit = ((INCOME_UNIT * selected_target.sum()) - EXPENSES) / 1000
        revenue.append(profit)
    revenue = pd.Series(revenue)
    mean = revenue.mean()#средняя прибыль
    lower_quantile = revenue.quantile(0.025)#0.025 квантиль
    upper_quantile = revenue.quantile(0.975)#0.975 квантиль
    loss_probability = len(revenue[revenue<0]) / len(revenue)
    return display('Средняя прибыль =', mean, '2.5% квантиль =', lower_quantile, '97.5% квантиль =', upper_quantile, 'вероятность убытков =', loss_probability)

С помощью функции выведем на экран значения средней прибыли, 95% доверительного интервала и риска убытков для каждого региона.

In [40]:
#выведем на экран значения средней прибыли, 95% доверительного интервала и риска убытков для каждого региона:
display('Регион 1:')
function (target_valid_1, predict_valid_1)
display('Регион 2:')
function (target_valid_2, predict_valid_2)
display('Регион 3:')
function (target_valid_3, predict_valid_3)

'Регион 1:'

'Средняя прибыль ='

409.1481639971147

'2.5% квантиль ='

-113.58844677278488

...

-69.50290035301158

'97.5% квантиль ='

981.5270708742662

'вероятность убытков ='

0.057

 **Вывод**
 
После оценки рисков мы можем оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. А это только один регион - регион 2. Хотелось бы отметить, что средняя прибыль по этому региону получилась выше, чем в остальных - 488 млн. руб. против 409 в первом и 429 млн. в третьем регионе. 

Таким образом, регион, который принесет наибольшую прибыль с минимальными рисками - это регион номер 2. Наша модель справилась с задачей, и мы можем дать рекомендации заказчику по бурению новых скважин в регионе 2. 

